In [1]:
import pandas as pd

In [84]:
data = pd.read_csv("/Users/aryagupta/Desktop/investor-pulse/data/reddit_posts.csv")

In [85]:
data.sample(5)

,title,selftext,author,created_utc,score,subreddit,tickers_mentioned
23,Is investing in stocks already heavily favored...,I left a comment elsewhere which after posting...,69philosopher,2025-05-10 20:19:59,16,investing,MSFT
8,GENIUS stablecoin and potential push for token...,/\* posted in /stocks but was removed due to o...,Moirailogist,2025-05-22 18:58:33,0,investing,"TSLA,AAPL,NVDA"
138,"Dow Jones, S&P 500 and Nasdaq trim losses as T...",U.S. stocks fell Friday and were on track to r...,Aluseda,2025-05-23 19:44:52,121,stocks,AAPL
32,Tesla Board Opened Search for a CEO to Succeed...,"With profits and stock price sinking, board me...",TechSMR2018,2025-05-01 01:19:09,1363,investing,TSLA
178,(05/1) Interesting Stocks Today - Unemployment...,Hi! I am an ex-prop shop equity trader. This i...,WinningWatchlist,2025-05-01 13:08:57,13,stocks,"TSLA,AAPL,MSFT,NVDA,AMZN"


In [86]:
data.isna().sum()

title                 0
selftext             12
author                0
created_utc           0
score                 0
subreddit             0
tickers_mentioned     0
dtype: int64

In [87]:
data.duplicated().sum()

np.int64(0)

In [88]:
data['created_utc'] = pd.to_datetime(data['created_utc'])
data['date'] = data['created_utc'].dt.date

In [89]:
data['full_text'] = (data['title'].fillna('') + ' ' + data['selftext'].fillna('')).str.lower()

In [90]:
import re

def clean_text(text):
    text = re.sub(r"http\S+|www\S+", "", text)         # URLs
    text = re.sub(r"u/\w+|r/\w+", "", text)            # Mentions
    text = re.sub(r"[^a-z\s]", "", text)               # Non-alpha
    text = re.sub(r"\s+", " ", text)                   # Extra spaces
    return text.strip()

data['clean_text'] = data['full_text'].apply(clean_text)

In [91]:
data.drop(['title', 'selftext', 'full_text', 'author'], axis = 1, inplace = True)

In [92]:
data[data['score'] > 5].sample(5)

,created_utc,score,subreddit,tickers_mentioned,date,clean_text
91,2025-05-16 22:05:05,32,wallstreetbets,NVDA,2025-05-16,holding nvda over weekend to be honest with th...
181,2025-04-30 20:08:32,329,stocks,MSFT,2025-04-30,microsoft shares jump on earnings revenue beat...
74,2025-05-24 15:26:17,652,wallstreetbets,NVDA,2025-05-24,i recently discovered nvda options are a hell ...
50,2025-05-30 00:23:13,41,wallstreetbets,NVDA,2025-05-30,nvda by july rd i believe this qualifies as a ...
51,2025-05-29 21:55:27,69,wallstreetbets,NVDA,2025-05-29,nvidias ai dominance is solid q data center bu...


In [93]:
data['tickers_mentioned'] = data['tickers_mentioned'].fillna('').str.replace(' ', '')
data = data[data['tickers_mentioned'] != '']  # Drop posts with no tickers

In [94]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/aryagupta/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [95]:
from nltk.sentiment import SentimentIntensityAnalyzer as sia

# Correct instantiation with parentheses
sia = SentimentIntensityAnalyzer()

# Apply with explicit text argument
data['sentiment_scores'] = data['clean_text'].apply(lambda text: sia.polarity_scores(str(text)))

In [96]:
data['compound'] = data['sentiment_scores'].apply(lambda x: x['compound'])

In [97]:
def classify_sentiment(compound):
    if compound >= 0.05:
        return 'positive'
    elif compound <= -0.05:
        return 'negative'
    else:
        return 'neutral'

data['sentiment'] = data['compound'].apply(classify_sentiment)

In [98]:
data.drop(['sentiment_scores', 'created_utc'], axis = 1, inplace = True)

In [99]:
data.sample(5)

,score,subreddit,tickers_mentioned,date,clean_text,compound,sentiment
118,0,stocks,NVDA,2025-06-03,voo etf vs safe growth big cap stocks hi guys ...,0.9921,positive
110,44,wallstreetbets,AMZN,2025-05-12,am i doing the china deal right nke and amzn c...,0.8201,positive
159,2,stocks,"AAPL,AMZN",2025-05-12,interesting stocks today major reductions in t...,0.8933,positive
151,15,stocks,AAPL,2025-05-16,daily discussion fundamentals friday may this ...,0.9730,positive
179,1,stocks,"MSFT,NVDA,AMZN",2025-05-01,inheriting a small fortune considering how to ...,0.9569,positive


In [100]:
data.to_csv('cleaned_reddit.csv')